# 在这之前，
1  优化G2第二步确定。d_ts = 0.35，现在优化zata，选择zata=10 or 50.
但是两个点肯定不够，现在扩充到4个点

In [22]:
fps_num = [32, 21, 16, 13, 11, 9, 8, 7, 7, 6, 6] # sum(fps_num) = 136
zata_all = [1, 10, 50, 100, 200, 400, 600, 800, 1000]
d_thetas = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
import io
import os
import re
import ase
import time
import numpy as np
import pandas as pd
from ase.io.trajectory import Trajectory

import os
import sys
import joblib
import datetime
import numpy as np
import pandas as pd
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [73]:
# select zata = 1 and 10 and 50 and 100 at d_ts = 0.35
zata_list = ['zata-1.csv', 'zata-10.csv', 'zata-50.csv', 'zata-100.csv','zata-200.csv', 
             'zata-400.csv', 'zata-600.csv', 'zata-800.csv', 'zata-1000.csv']
pwd_ = '/Users/renzhe/Desktop/project2/g2/zata_all/dataall/'
data_zata_50 = pd.read_csv(pwd_+zata_list[2])
data_inf = data_zata_50.iloc[:,[0,1]]

#======================================================================================
zata = [1, 10, 50, 100, 200]
zata_list = ['zata-1.csv', 'zata-10.csv', 'zata-50.csv','zata-100.csv', 'zata-200.csv']
pwd_ = '/Users/renzhe/Desktop/project2/g2/zata_all/dataall/'

l_name = []
for e in zata:
    for i in range(9):
        l_n = '{}_ts_0.35/{}'.format(e,i)
        l_name.append(l_n)

n_ = 95        
data_035 = pd.DataFrame()
for z_l in zata_list:
    path_i = pwd_ + z_l
    df_i = pd.read_csv(path_i)
    d035 = df_i.iloc[:, n_:n_+9]
    data_035 = pd.concat([data_035, d035], axis=1)
    '''
       len(data_035) = 2 + 9 * 5 = 47 (2image序号、Y_label = Force；
       9 * 4是d_ts = 0.35时有9个g2，eta选择1和10和50和100和200共5组)。
    '''
    
data_035.columns = l_name  
data_035 = pd.concat([data_inf, data_035], axis=1)
# data_035.to_csv(pwd_+'optimize_zata_035.csv', index=False)

In [74]:
data_035.shape

(112968, 47)

In [75]:
data_035

,Img | Atom,Force,1_ts_0.35/0,1_ts_0.35/1,1_ts_0.35/2,1_ts_0.35/3,1_ts_0.35/4,1_ts_0.35/5,1_ts_0.35/6,1_ts_0.35/7,...,100_ts_0.35/8,200_ts_0.35/0,200_ts_0.35/1,200_ts_0.35/2,200_ts_0.35/3,200_ts_0.35/4,200_ts_0.35/5,200_ts_0.35/6,200_ts_0.35/7,200_ts_0.35/8
0,0 | 0,0.265362,0.041094,0.040500,0.040058,0.039732,0.039495,0.039343,0.039298,0.039535,...,1.795715e-07,1.050057e-07,0.026600,0.066974,0.007919,0.004722,0.088944,0.076601,8.200814e-05,5.195879e-13
1,0 | 1,0.339377,0.032166,0.031985,0.031909,0.031924,0.032022,0.032219,0.032555,0.033233,...,6.526385e-06,4.410290e-07,0.040381,0.039038,0.005739,0.006208,0.107614,0.058968,1.818194e-03,1.414039e-09
2,0 | 2,0.309006,0.041509,0.040774,0.040143,0.039583,0.039060,0.038548,0.038028,0.037596,...,1.216391e-05,1.957316e-07,0.020631,0.073725,0.007464,0.008972,0.094098,0.016732,2.866483e-03,5.070699e-09
3,0 | 3,0.618771,0.042041,0.041642,0.041367,0.041193,0.041102,0.041097,0.041208,0.041617,...,3.679189e-07,4.826438e-08,0.023986,0.067239,0.006208,0.011634,0.114292,0.080650,1.615350e-04,2.864873e-12
4,0 | 4,0.521276,0.040675,0.040429,0.040290,0.040244,0.040280,0.040410,0.040669,0.041252,...,1.601380e-05,3.738793e-06,0.037584,0.065982,0.006055,0.009230,0.130796,0.041696,3.220435e-03,1.130349e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112963,1224 | 139,0.672515,0.123244,0.120850,0.118627,0.116500,0.114351,0.112058,0.109466,0.106450,...,1.071628e-11,4.729453e-05,0.196215,0.118508,0.057423,0.058383,0.095224,0.007209,1.234650e-09,2.512123e-22
112964,1224 | 140,0.729798,0.118314,0.115662,0.113264,0.111016,0.108790,0.106465,0.103898,0.101000,...,1.803010e-09,8.830096e-06,0.196406,0.130543,0.034273,0.191709,0.023960,0.040864,5.855599e-07,1.603100e-17
112965,1224 | 141,1.130905,0.123199,0.121500,0.119849,0.118219,0.116526,0.114667,0.112502,0.109907,...,1.541359e-13,6.188255e-02,0.169079,0.107743,0.107044,0.039746,0.102222,0.000366,9.710805e-12,2.981690e-25
112966,1224 | 142,1.540307,0.129305,0.127967,0.126743,0.125611,0.124505,0.123365,0.122127,0.120811,...,1.519735e-15,2.749496e-04,0.342535,0.094461,0.040293,0.148733,0.123188,0.000024,7.311399e-14,5.814832e-27


In [86]:
data035_ = data_035
from collections import Counter
def GBDT_opt_g2_zata(random_state_1, random_state_2, n_ks_=1000, data035=data035_):
    x1w = data035.drop(['Img   |   Atom', 'Force'], axis=1)
    y1w = data035['Force'].values
    datank = data035.sample(n=n_ks_, replace=False, random_state=random_state_1, axis=0)
    xnk = datank.drop(['Img   |   Atom', 'Force'], axis=1)
    ynk = datank['Force'].values
    x_train, x_test, y_train, y_test = train_test_split(xnk,ynk,random_state=random_state_2,train_size=0.99)
    gbr = GradientBoostingRegressor(n_estimators=20, max_depth=8, min_samples_split=3, learning_rate=0.1, loss='squared_error')
    gbr.fit(x_train, y_train.ravel())
    y_gbr = gbr.predict(x_train)
    mse = mean_squared_error(y_train, y_gbr)
    rr = np.sqrt(mse)
    y_gbr1 = gbr.predict(x_test)
    r2 = r2_score(y_test, y_gbr1)
#     if r2 > 0.8 and rr < 0.15:
#         joblib.dump(gbr, '/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata/zata_{}_{}.m'.
#                     format(random_state_1, random_state_2))
#         print("RMSE:", rr)
#         print("  R2:", r2)
#         feature_importance = gbr.feature_importances_
# #         print(feature_importance.shape)
#         ind_zata = [1, 10, 50, 100, 200]
#         df1 = pd.DataFrame(columns=data035.columns[2:], index=ind_zata)
#         df1.iloc[0, 0:9]  = feature_importance[0:9]
#         df1.iloc[1, 9:18] = feature_importance[9:18]
#         df1.iloc[2, 18:27] = feature_importance[18:27]
#         df1.iloc[3, 27:36] = feature_importance[27:36]
#         df1.iloc[4, 36:45] = feature_importance[36:45]
        
#         df1 = df1.fillna(0)
#         a0  = np.sum(feature_importance[0:9])      #1 
#         a1  = np.sum(feature_importance[9:18])     #10
#         a2  = np.sum(feature_importance[18:27])    #50 
#         a3  = np.sum(feature_importance[27:36])    #100 
#         a4  = np.sum(feature_importance[36:45])    #200 

        

    feature_importance = gbr.feature_importances_
    ind_zata = [1, 10, 50, 100, 200]
    df1 = pd.DataFrame(columns=data035.columns[2:], index=ind_zata)
    df1.iloc[0, 0:9]  = feature_importance[0:9]
    df1.iloc[1, 9:18] = feature_importance[9:18]
    df1.iloc[2, 18:27] = feature_importance[18:27]
    df1.iloc[3, 27:36] = feature_importance[27:36]
    df1.iloc[4, 36:45] = feature_importance[36:45]

    df1 = df1.fillna(0)
    a0  = np.sum(feature_importance[0:9])      #1 
    a1  = np.sum(feature_importance[9:18])     #10
    a2  = np.sum(feature_importance[18:27])    #50 
    a3  = np.sum(feature_importance[27:36])    #100 
    a4  = np.sum(feature_importance[36:45])    #200 
    
    aa = np.reshape(feature_importance, (5, 9))
    dd = pd.DataFrame(aa, index=[1, 10, 50, 100, 200], columns=range(9))
    nn = np.argmax(aa, axis=0)
    res = [ind_zata[n] for n in nn]
    element_count = Counter(res)
    sst1 = (element_count[200] < element_count[10] and element_count[200] < element_count[50])
    sst2 = (element_count[100] < element_count[10] and element_count[100] < element_count[50])
    st1 = (a2 > a0 and a2 > a3 and a2 > a4)
    st2 = (a1 > a0 and a1 > a3 and a1 > a4)
#     if st1 or st2:
    if sst1 and sst2:
        print("RMSE:", rr)
        print("  R2:", r2)
        joblib.dump(gbr, '/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata/zata_{}_{}.m'.
                format(random_state_1, random_state_2))
    print('Finish random_s1-s2: {}-{}'.format(random_state_1,random_state_2))
    return gbr

In [87]:
for i in range(0, 100):
    for j in range(0, 100):
        GBDT_opt_g2_zata(random_state_1=j, random_state_2=i)

Finish random_s1-s2: 0-0
Finish random_s1-s2: 1-0
Finish random_s1-s2: 2-0
Finish random_s1-s2: 3-0
Finish random_s1-s2: 4-0
Finish random_s1-s2: 5-0
Finish random_s1-s2: 6-0
Finish random_s1-s2: 7-0
Finish random_s1-s2: 8-0
Finish random_s1-s2: 9-0
Finish random_s1-s2: 10-0
Finish random_s1-s2: 11-0
Finish random_s1-s2: 12-0
Finish random_s1-s2: 13-0
Finish random_s1-s2: 14-0
Finish random_s1-s2: 15-0
Finish random_s1-s2: 16-0
Finish random_s1-s2: 17-0
Finish random_s1-s2: 18-0
Finish random_s1-s2: 19-0
Finish random_s1-s2: 20-0
Finish random_s1-s2: 21-0
Finish random_s1-s2: 22-0
Finish random_s1-s2: 23-0
Finish random_s1-s2: 24-0
Finish random_s1-s2: 25-0
Finish random_s1-s2: 26-0
Finish random_s1-s2: 27-0
Finish random_s1-s2: 28-0
Finish random_s1-s2: 29-0
Finish random_s1-s2: 30-0
Finish random_s1-s2: 31-0
Finish random_s1-s2: 32-0
Finish random_s1-s2: 33-0
Finish random_s1-s2: 34-0
Finish random_s1-s2: 35-0
Finish random_s1-s2: 36-0
Finish random_s1-s2: 37-0
Finish random_s1-s2: 3

Finish random_s1-s2: 15-3
Finish random_s1-s2: 16-3
Finish random_s1-s2: 17-3
Finish random_s1-s2: 18-3
Finish random_s1-s2: 19-3
Finish random_s1-s2: 20-3
Finish random_s1-s2: 21-3
Finish random_s1-s2: 22-3
Finish random_s1-s2: 23-3
Finish random_s1-s2: 24-3
Finish random_s1-s2: 25-3
RMSE: 0.1615307716709986
  R2: 0.606563555289972
Finish random_s1-s2: 26-3
Finish random_s1-s2: 27-3
Finish random_s1-s2: 28-3
Finish random_s1-s2: 29-3
Finish random_s1-s2: 30-3
Finish random_s1-s2: 31-3
Finish random_s1-s2: 32-3
Finish random_s1-s2: 33-3
Finish random_s1-s2: 34-3
Finish random_s1-s2: 35-3
Finish random_s1-s2: 36-3
Finish random_s1-s2: 37-3
Finish random_s1-s2: 38-3
Finish random_s1-s2: 39-3
Finish random_s1-s2: 40-3
Finish random_s1-s2: 41-3
Finish random_s1-s2: 42-3
Finish random_s1-s2: 43-3
Finish random_s1-s2: 44-3
Finish random_s1-s2: 45-3
Finish random_s1-s2: 46-3
Finish random_s1-s2: 47-3
Finish random_s1-s2: 48-3
Finish random_s1-s2: 49-3
Finish random_s1-s2: 50-3
Finish random_s

Finish random_s1-s2: 30-6
Finish random_s1-s2: 31-6
Finish random_s1-s2: 32-6
Finish random_s1-s2: 33-6
Finish random_s1-s2: 34-6
Finish random_s1-s2: 35-6
Finish random_s1-s2: 36-6
Finish random_s1-s2: 37-6
Finish random_s1-s2: 38-6
Finish random_s1-s2: 39-6
Finish random_s1-s2: 40-6
Finish random_s1-s2: 41-6
Finish random_s1-s2: 42-6
RMSE: 0.18840055476071046
  R2: 0.13029578246339102
Finish random_s1-s2: 43-6
Finish random_s1-s2: 44-6
Finish random_s1-s2: 45-6
Finish random_s1-s2: 46-6
Finish random_s1-s2: 47-6
Finish random_s1-s2: 48-6
Finish random_s1-s2: 49-6
Finish random_s1-s2: 50-6
Finish random_s1-s2: 51-6
Finish random_s1-s2: 52-6
Finish random_s1-s2: 53-6
Finish random_s1-s2: 54-6
Finish random_s1-s2: 55-6
Finish random_s1-s2: 56-6
Finish random_s1-s2: 57-6
Finish random_s1-s2: 58-6
Finish random_s1-s2: 59-6
Finish random_s1-s2: 60-6
Finish random_s1-s2: 61-6
Finish random_s1-s2: 62-6
Finish random_s1-s2: 63-6
Finish random_s1-s2: 64-6
Finish random_s1-s2: 65-6
Finish rando

Finish random_s1-s2: 43-9
Finish random_s1-s2: 44-9
Finish random_s1-s2: 45-9
Finish random_s1-s2: 46-9
Finish random_s1-s2: 47-9
Finish random_s1-s2: 48-9
Finish random_s1-s2: 49-9
Finish random_s1-s2: 50-9
Finish random_s1-s2: 51-9
Finish random_s1-s2: 52-9
Finish random_s1-s2: 53-9
Finish random_s1-s2: 54-9
Finish random_s1-s2: 55-9
Finish random_s1-s2: 56-9
Finish random_s1-s2: 57-9
Finish random_s1-s2: 58-9
Finish random_s1-s2: 59-9
Finish random_s1-s2: 60-9
Finish random_s1-s2: 61-9
Finish random_s1-s2: 62-9
Finish random_s1-s2: 63-9
Finish random_s1-s2: 64-9
Finish random_s1-s2: 65-9
Finish random_s1-s2: 66-9
Finish random_s1-s2: 67-9
Finish random_s1-s2: 68-9
Finish random_s1-s2: 69-9
Finish random_s1-s2: 70-9
Finish random_s1-s2: 71-9
Finish random_s1-s2: 72-9
Finish random_s1-s2: 73-9
Finish random_s1-s2: 74-9
Finish random_s1-s2: 75-9
Finish random_s1-s2: 76-9
Finish random_s1-s2: 77-9
Finish random_s1-s2: 78-9
Finish random_s1-s2: 79-9
Finish random_s1-s2: 80-9
Finish rando

Finish random_s1-s2: 48-12
Finish random_s1-s2: 49-12
Finish random_s1-s2: 50-12
Finish random_s1-s2: 51-12
Finish random_s1-s2: 52-12
Finish random_s1-s2: 53-12
Finish random_s1-s2: 54-12
Finish random_s1-s2: 55-12
Finish random_s1-s2: 56-12
Finish random_s1-s2: 57-12
Finish random_s1-s2: 58-12
Finish random_s1-s2: 59-12
Finish random_s1-s2: 60-12
Finish random_s1-s2: 61-12
Finish random_s1-s2: 62-12
Finish random_s1-s2: 63-12
Finish random_s1-s2: 64-12
Finish random_s1-s2: 65-12
Finish random_s1-s2: 66-12
Finish random_s1-s2: 67-12
Finish random_s1-s2: 68-12
Finish random_s1-s2: 69-12
Finish random_s1-s2: 70-12
Finish random_s1-s2: 71-12
Finish random_s1-s2: 72-12
Finish random_s1-s2: 73-12
Finish random_s1-s2: 74-12
Finish random_s1-s2: 75-12
Finish random_s1-s2: 76-12
Finish random_s1-s2: 77-12
Finish random_s1-s2: 78-12
Finish random_s1-s2: 79-12
Finish random_s1-s2: 80-12
Finish random_s1-s2: 81-12
Finish random_s1-s2: 82-12
Finish random_s1-s2: 83-12
Finish random_s1-s2: 84-12
F

Finish random_s1-s2: 53-15
Finish random_s1-s2: 54-15
Finish random_s1-s2: 55-15
Finish random_s1-s2: 56-15
Finish random_s1-s2: 57-15
Finish random_s1-s2: 58-15
Finish random_s1-s2: 59-15
Finish random_s1-s2: 60-15
Finish random_s1-s2: 61-15
Finish random_s1-s2: 62-15
Finish random_s1-s2: 63-15
Finish random_s1-s2: 64-15
Finish random_s1-s2: 65-15
Finish random_s1-s2: 66-15
Finish random_s1-s2: 67-15
Finish random_s1-s2: 68-15
Finish random_s1-s2: 69-15
Finish random_s1-s2: 70-15
Finish random_s1-s2: 71-15
Finish random_s1-s2: 72-15
Finish random_s1-s2: 73-15
Finish random_s1-s2: 74-15
Finish random_s1-s2: 75-15
Finish random_s1-s2: 76-15
Finish random_s1-s2: 77-15
Finish random_s1-s2: 78-15
Finish random_s1-s2: 79-15
Finish random_s1-s2: 80-15
Finish random_s1-s2: 81-15
Finish random_s1-s2: 82-15
Finish random_s1-s2: 83-15
Finish random_s1-s2: 84-15
Finish random_s1-s2: 85-15
Finish random_s1-s2: 86-15
Finish random_s1-s2: 87-15
Finish random_s1-s2: 88-15
Finish random_s1-s2: 89-15
F

KeyboardInterrupt: 

In [94]:
data035_ = data_035
def im_a(path_s1_s2):
    gbr_035 = joblib.load(path_s1_s2)
    feature_importance = gbr_035.feature_importances_
    ind_zata = [1, 10, 50, 100, 200]
    df1 = pd.DataFrame(columns=data035_.columns[2:], index=ind_zata)
    df1.iloc[0, 0:9]  = feature_importance[0:9]
    df1.iloc[1, 9:18] = feature_importance[9:18]
    df1.iloc[2, 18:27] = feature_importance[18:27]
    df1.iloc[3, 27:36] = feature_importance[27:36]
    df1.iloc[4, 36:45] = feature_importance[36:45]
    df1 = df1.fillna(0)
    a0  = np.sum(feature_importance[0:9])
    a1  = np.sum(feature_importance[9:18])
    a2  = np.sum(feature_importance[18:27])
    a3  = np.sum(feature_importance[27:36])
    a4  = np.sum(feature_importance[36:45])
    feature_importance = gbr_035.feature_importances_
    aa = np.reshape(feature_importance, (5, 9))
    dd = pd.DataFrame(aa, index=[1, 10, 50, 100, 200], columns=range(9))
    nn = np.argmax(aa, axis=0)
    res = [ind_zata[n] for n in nn]
    return res

im_a(path_s1_s2='/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata/zata_72_10.m')

[50, 200, 200, 50, 50, 10, 10, 50, 10]

In [99]:
data035_ = data_035
def im_a(path_s1_s2):
    gbr_035 = joblib.load(path_s1_s2)
    feature_importance = gbr_035.feature_importances_
    ind_zata = [1, 10, 50, 100, 200]
    df1 = pd.DataFrame(columns=data035_.columns[2:], index=ind_zata)
    df1.iloc[0, 0:9]  = feature_importance[0:9]
    df1.iloc[1, 9:18] = feature_importance[9:18]
    df1.iloc[2, 18:27] = feature_importance[18:27]
    df1.iloc[3, 27:36] = feature_importance[27:36]
    df1.iloc[4, 36:45] = feature_importance[36:45]
    df1.to_csv('/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata/importance_g2.csv')
    df1 = df1.fillna(0)
    a0  = np.sum(feature_importance[0:9])
    a1  = np.sum(feature_importance[9:18])
    a2  = np.sum(feature_importance[18:27])
    a3  = np.sum(feature_importance[27:36])
    a4  = np.sum(feature_importance[36:45])
    feature_importance = gbr_035.feature_importances_
    aa = np.reshape(feature_importance, (5, 9))
    dd = pd.DataFrame(aa, index=[1, 10, 50, 100, 200], columns=range(9))
    nn = np.argmax(aa, axis=0)
    res = [ind_zata[n] for n in nn]
    return res

im_a(path_s1_s2='/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata/recal5.m')

[10, 200, 50, 100, 50, 10, 50, 50, 50]

In [ ]:
#important 分析
import os
def list_files_in_folder(folder_path='/Users/renzhe/Desktop/project2/g2/zata_all/model/opt-zata/goon_zata'):
    l_ = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            l_.append(file_path)
    return l_